In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D, Convolution2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Flatten, Dropout
import math
import numpy as np
import os
from keras.datasets import mnist
from keras.optimizers import Adam
from PIL import Image
from keras import backend as K
K.set_image_dim_ordering('th')

BATCH_SIZE = 32
NUM_EPOCH = 20
GENERATED_IMAGE_PATH = 'generated_images/' # 生成画像の保存先

def generator_model():
    model = Sequential()
    model.add(Dense(input_dim=100, output_dim=1024))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(128*7*7))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Reshape((128, 7, 7), input_shape=(128*7*7,)))
    model.add(UpSampling2D((2, 2)))
    model.add(Convolution2D(64, 5, 5, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2)))
    model.add(Convolution2D(1, 5, 5, border_mode='same'))
    model.add(Activation('tanh'))
    return model

def discriminator_model():
    model = Sequential()
    model.add(Convolution2D(64, 5, 5,
                            subsample=(2, 2),
                            border_mode='same',
                            input_shape=(1, 28, 28)))
    model.add(LeakyReLU(0.2))
    model.add(Convolution2D(128, 5, 5, subsample=(2, 2)))
    model.add(LeakyReLU(0.2))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

def combine_images(generated_images):
    total = generated_images.shape[0]
    cols = int(math.sqrt(total))
    rows = math.ceil(float(total)/cols)
    width, height = generated_images.shape[2:]
    combined_image = np.zeros((height*rows, width*cols),
                              dtype=generated_images.dtype)

    for index, image in enumerate(generated_images):
        i = int(index/cols)
        j = index % cols
        combined_image[width*i:width*(i+1), height*j:height*(j+1)] = image[0, :, :]
    return combined_image

def train():
    (X_train, y_train), (_, _) = mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1], X_train.shape[2])

    discriminator = discriminator_model()
    d_opt = Adam(lr=1e-5, beta_1=0.1)
    discriminator.compile(loss='binary_crossentropy', optimizer=d_opt)

    # generator+discriminator （discriminator部分の重みは固定）
    discriminator.trainable = False
    generator = generator_model()
    dcgan = Sequential([generator, discriminator])
    g_opt = Adam(lr=2e-4, beta_1=0.5)
    dcgan.compile(loss='binary_crossentropy', optimizer=g_opt)

    num_batches = int(X_train.shape[0] / BATCH_SIZE)
    print('Number of batches:', num_batches)
    for epoch in range(NUM_EPOCH):

        for index in range(num_batches):
            noise = np.array([np.random.uniform(-1, 1, 100) for _ in range(BATCH_SIZE)])
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            generated_images = generator.predict(noise, verbose=0)

            # 生成画像を出力
            if index % 500 == 0:
                image = combine_images(generated_images)
                image = image*127.5 + 127.5
                if not os.path.exists(GENERATED_IMAGE_PATH):
                    os.mkdir(GENERATED_IMAGE_PATH)
                Image.fromarray(image.astype(np.uint8))\
                    .save(GENERATED_IMAGE_PATH+"%04d_%04d.png" % (epoch, index))

            # discriminatorを更新
            X = np.concatenate((image_batch, generated_images))
            y = [1]*BATCH_SIZE + [0]*BATCH_SIZE
            d_loss = discriminator.train_on_batch(X, y)

            # generatorを更新
            noise = np.array([np.random.uniform(-1, 1, 100) for _ in range(BATCH_SIZE)])
            g_loss = dcgan.train_on_batch(noise, [1]*BATCH_SIZE)
            print("epoch: %d, batch: %d, g_loss: %f, d_loss: %f" % (epoch, index, g_loss, d_loss))

        generator.save_weights('generator.h5')
        discriminator.save_weights('discriminator.h5')
        
train()

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), input_shape=(1, 28, 28..., strides=(2, 2), padding="same")`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), strides=(2, 2))`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, units=1024)`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `C

Number of batches: 1875


C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch: 0, batch: 0, g_loss: 0.688306, d_loss: 0.674798
epoch: 0, batch: 1, g_loss: 0.694943, d_loss: 0.676143
epoch: 0, batch: 2, g_loss: 0.666119, d_loss: 0.681563
epoch: 0, batch: 3, g_loss: 0.647440, d_loss: 0.697943
epoch: 0, batch: 4, g_loss: 0.629723, d_loss: 0.689374
epoch: 0, batch: 5, g_loss: 0.645312, d_loss: 0.696380
epoch: 0, batch: 6, g_loss: 0.645398, d_loss: 0.688990
epoch: 0, batch: 7, g_loss: 0.660950, d_loss: 0.684946
epoch: 0, batch: 8, g_loss: 0.658573, d_loss: 0.691400
epoch: 0, batch: 9, g_loss: 0.655464, d_loss: 0.697176
epoch: 0, batch: 10, g_loss: 0.660153, d_loss: 0.692060
epoch: 0, batch: 11, g_loss: 0.665098, d_loss: 0.679089
epoch: 0, batch: 12, g_loss: 0.666447, d_loss: 0.685840
epoch: 0, batch: 13, g_loss: 0.668605, d_loss: 0.678361
epoch: 0, batch: 14, g_loss: 0.655659, d_loss: 0.686813
epoch: 0, batch: 15, g_loss: 0.663777, d_loss: 0.677146
epoch: 0, batch: 16, g_loss: 0.651913, d_loss: 0.672667
epoch: 0, batch: 17, g_loss: 0.640673, d_loss: 0.676377
ep

epoch: 0, batch: 146, g_loss: 0.610524, d_loss: 0.700368
epoch: 0, batch: 147, g_loss: 0.610020, d_loss: 0.693715
epoch: 0, batch: 148, g_loss: 0.613637, d_loss: 0.700577
epoch: 0, batch: 149, g_loss: 0.609679, d_loss: 0.694689
epoch: 0, batch: 150, g_loss: 0.626651, d_loss: 0.705221
epoch: 0, batch: 151, g_loss: 0.609599, d_loss: 0.697748
epoch: 0, batch: 152, g_loss: 0.604009, d_loss: 0.700126
epoch: 0, batch: 153, g_loss: 0.616896, d_loss: 0.694490
epoch: 0, batch: 154, g_loss: 0.608085, d_loss: 0.699112
epoch: 0, batch: 155, g_loss: 0.612829, d_loss: 0.696875
epoch: 0, batch: 156, g_loss: 0.612125, d_loss: 0.695462
epoch: 0, batch: 157, g_loss: 0.618897, d_loss: 0.689512
epoch: 0, batch: 158, g_loss: 0.624283, d_loss: 0.701032
epoch: 0, batch: 159, g_loss: 0.612521, d_loss: 0.699608
epoch: 0, batch: 160, g_loss: 0.616624, d_loss: 0.698566
epoch: 0, batch: 161, g_loss: 0.619897, d_loss: 0.698997
epoch: 0, batch: 162, g_loss: 0.617283, d_loss: 0.700814
epoch: 0, batch: 163, g_loss: 0

...

epoch: 19, batch: 1301, g_loss: 0.659746, d_loss: 0.689026
epoch: 19, batch: 1302, g_loss: 0.646861, d_loss: 0.686853
epoch: 19, batch: 1303, g_loss: 0.657714, d_loss: 0.687821
epoch: 19, batch: 1304, g_loss: 0.665504, d_loss: 0.695962
epoch: 19, batch: 1305, g_loss: 0.653475, d_loss: 0.686148
epoch: 19, batch: 1306, g_loss: 0.673290, d_loss: 0.685659
epoch: 19, batch: 1307, g_loss: 0.666362, d_loss: 0.695303
epoch: 19, batch: 1308, g_loss: 0.703716, d_loss: 0.699827
epoch: 19, batch: 1309, g_loss: 0.716919, d_loss: 0.694785
epoch: 19, batch: 1310, g_loss: 0.734826, d_loss: 0.693870
epoch: 19, batch: 1311, g_loss: 0.751138, d_loss: 0.679410
epoch: 19, batch: 1312, g_loss: 0.762925, d_loss: 0.675209
epoch: 19, batch: 1313, g_loss: 0.763591, d_loss: 0.691010
epoch: 19, batch: 1314, g_loss: 0.752081, d_loss: 0.693718
epoch: 19, batch: 1315, g_loss: 0.738253, d_loss: 0.685758
epoch: 19, batch: 1316, g_loss: 0.721773, d_loss: 0.698716
epoch: 19, batch: 1317, g_loss: 0.722463, d_loss: 0.6867

epoch: 19, batch: 1441, g_loss: 0.665093, d_loss: 0.698638
epoch: 19, batch: 1442, g_loss: 0.683326, d_loss: 0.687896
epoch: 19, batch: 1443, g_loss: 0.685708, d_loss: 0.692136
epoch: 19, batch: 1444, g_loss: 0.682804, d_loss: 0.693963
epoch: 19, batch: 1445, g_loss: 0.702356, d_loss: 0.693697
epoch: 19, batch: 1446, g_loss: 0.687855, d_loss: 0.690053
epoch: 19, batch: 1447, g_loss: 0.699799, d_loss: 0.692068
epoch: 19, batch: 1448, g_loss: 0.710600, d_loss: 0.691978
epoch: 19, batch: 1449, g_loss: 0.730741, d_loss: 0.683267
epoch: 19, batch: 1450, g_loss: 0.722741, d_loss: 0.695220
epoch: 19, batch: 1451, g_loss: 0.722648, d_loss: 0.689595
epoch: 19, batch: 1452, g_loss: 0.726187, d_loss: 0.686334
epoch: 19, batch: 1453, g_loss: 0.713263, d_loss: 0.697121
epoch: 19, batch: 1454, g_loss: 0.722144, d_loss: 0.692087
epoch: 19, batch: 1455, g_loss: 0.716258, d_loss: 0.692689
epoch: 19, batch: 1456, g_loss: 0.704140, d_loss: 0.693572
epoch: 19, batch: 1457, g_loss: 0.691115, d_loss: 0.6898

epoch: 19, batch: 1580, g_loss: 0.711151, d_loss: 0.694215
epoch: 19, batch: 1581, g_loss: 0.713087, d_loss: 0.695603
epoch: 19, batch: 1582, g_loss: 0.726498, d_loss: 0.686140
epoch: 19, batch: 1583, g_loss: 0.724025, d_loss: 0.696711
epoch: 19, batch: 1584, g_loss: 0.723086, d_loss: 0.688444
epoch: 19, batch: 1585, g_loss: 0.722864, d_loss: 0.690675
epoch: 19, batch: 1586, g_loss: 0.733163, d_loss: 0.696509
epoch: 19, batch: 1587, g_loss: 0.736449, d_loss: 0.693428
epoch: 19, batch: 1588, g_loss: 0.719032, d_loss: 0.690732
epoch: 19, batch: 1589, g_loss: 0.725627, d_loss: 0.692045
epoch: 19, batch: 1590, g_loss: 0.735917, d_loss: 0.683470
epoch: 19, batch: 1591, g_loss: 0.717551, d_loss: 0.694949
epoch: 19, batch: 1592, g_loss: 0.708077, d_loss: 0.696383
epoch: 19, batch: 1593, g_loss: 0.704202, d_loss: 0.695183
epoch: 19, batch: 1594, g_loss: 0.697341, d_loss: 0.694093
epoch: 19, batch: 1595, g_loss: 0.687055, d_loss: 0.696193
epoch: 19, batch: 1596, g_loss: 0.689769, d_loss: 0.6893

epoch: 19, batch: 1718, g_loss: 0.668364, d_loss: 0.690671
epoch: 19, batch: 1719, g_loss: 0.675505, d_loss: 0.691983
epoch: 19, batch: 1720, g_loss: 0.689168, d_loss: 0.685912
epoch: 19, batch: 1721, g_loss: 0.686745, d_loss: 0.694371
epoch: 19, batch: 1722, g_loss: 0.683508, d_loss: 0.689908
epoch: 19, batch: 1723, g_loss: 0.705559, d_loss: 0.691181
epoch: 19, batch: 1724, g_loss: 0.705268, d_loss: 0.689518
epoch: 19, batch: 1725, g_loss: 0.700075, d_loss: 0.692633
epoch: 19, batch: 1726, g_loss: 0.718280, d_loss: 0.694819
epoch: 19, batch: 1727, g_loss: 0.715708, d_loss: 0.687676
epoch: 19, batch: 1728, g_loss: 0.700100, d_loss: 0.696437
epoch: 19, batch: 1729, g_loss: 0.700060, d_loss: 0.689707
epoch: 19, batch: 1730, g_loss: 0.698897, d_loss: 0.688045
epoch: 19, batch: 1731, g_loss: 0.702808, d_loss: 0.693581
epoch: 19, batch: 1732, g_loss: 0.700207, d_loss: 0.684425
epoch: 19, batch: 1733, g_loss: 0.710840, d_loss: 0.696766
epoch: 19, batch: 1734, g_loss: 0.697860, d_loss: 0.6885

epoch: 19, batch: 1861, g_loss: 0.683906, d_loss: 0.682802
epoch: 19, batch: 1862, g_loss: 0.681698, d_loss: 0.685179
epoch: 19, batch: 1863, g_loss: 0.663542, d_loss: 0.683562
epoch: 19, batch: 1864, g_loss: 0.659890, d_loss: 0.688861
epoch: 19, batch: 1865, g_loss: 0.656874, d_loss: 0.684709
epoch: 19, batch: 1866, g_loss: 0.663773, d_loss: 0.684916
epoch: 19, batch: 1867, g_loss: 0.663954, d_loss: 0.683812
epoch: 19, batch: 1868, g_loss: 0.669101, d_loss: 0.682818
epoch: 19, batch: 1869, g_loss: 0.671418, d_loss: 0.695411
epoch: 19, batch: 1870, g_loss: 0.685588, d_loss: 0.689176
epoch: 19, batch: 1871, g_loss: 0.698098, d_loss: 0.688834
epoch: 19, batch: 1872, g_loss: 0.702258, d_loss: 0.675404
epoch: 19, batch: 1873, g_loss: 0.687226, d_loss: 0.679264
epoch: 19, batch: 1874, g_loss: 0.695759, d_loss: 0.679179
